In [1]:
import sys
import os
import datetime as dt
import math
import matplotlib as plt

sys.path.append("D:\Desktop\Github\Trading_bot")
from Crawler.functions import *

In [20]:
# # 做多: 當(今天短週期均線 > 今天長週期均線) 且 (昨天短週期均線 < 昨天長週期均線)
# # 做空: 當(今天短週期均線 < 今天長週期均線) 且 (昨天短週期均線 > 昨天長週期均線)
# signal_long = (sma1 > sma2) & (sma1.shift() < sma2.shift())
# signal_short = (sma1 < sma2) & (sma1.shift() > sma2.shift())

# signal_long.astype(int).plot()
# (-signal_short.astype(int)).plot()

# # combine long and short signal
# signal = signal_long.copy()
# signal[signal_short] = -1

In [2]:
from backtesting import Backtest
from backtesting.lib import SignalStrategy
import pandas as pd

In [3]:
coin_txt = open("../../Crawler/coin.txt","r",encoding = 'utf8')
coin = coin_txt.read()
coin_txt.close()

DB = DB_table(db_name = "../../Crawler/test.db", coin_name = coin)
DB.init_db()

df = DB.get_all()
df.set_index('Timestamp', inplace=True)

re_df = df["2021-09-13 23:00:00":]
print(re_df)

                         Open      High       Low     Close        Volume
Timestamp                                                                
2017-08-17 12:00:00   4261.48   4313.62   4261.32   4308.83     47.181009
2017-08-17 13:00:00   4308.83   4328.69   4291.37   4315.32     23.234916
2017-08-17 14:00:00   4330.29   4345.45   4309.37   4324.35      7.229691
2017-08-17 15:00:00   4316.62   4349.99   4287.41   4349.99      4.443249
2017-08-17 16:00:00   4333.32   4377.85   4333.32   4360.69      0.972807
...                       ...       ...       ...       ...           ...
2022-10-13 19:00:00  18689.44  18831.07  18658.35  18753.20  13934.924310
2022-10-13 20:00:00  18754.09  18777.72  18742.25  18756.26    919.259730
2022-10-13 21:00:00  18385.30  18479.96  18273.00  18438.18  27029.805740
2022-10-13 22:00:00  18430.95  18436.38  18400.00  18432.69   1868.793230
2022-10-13 23:00:00  18415.65  18832.60  18403.65  18751.54  14220.897600

[45077 rows x 5 columns]


In [10]:
class Strategy(SignalStrategy):
    def init(self):
        super().init()
        
        # Precompute the two moving averages
        close = pd.Series(self.data.Close)
        sma1 = close.rolling(20).mean()
        sma2 = close.rolling(60).mean()
        
        # Precompute signal
        signal_long = (sma1 > sma2) & (sma1.shift() < sma2.shift())
        signal_short = (sma1 < sma2) & (sma1.shift() > sma2.shift())
        
        signal = signal_long
        signal[signal_short] = -1
        
        self.set_signal(signal)
        
        
    def next(self):
        super().next()

In [19]:
bt = Backtest(re_df, Strategy, cash=1000000)
bt.run()
bt.plot()

Row(id='5149', ...)